In [95]:
import pandas as pd
import final_project.utils.paths as path
import numpy
import talib

Cargamos los datos desde el 2014, los datos descargados desde Interactive Broker son en temporalidad de 30 minutos

In [96]:
df1=pd.read_csv(path.data_raw_dir("SPY2014.csv"), parse_dates=[0])
df2=pd.read_csv(path.data_raw_dir("SPY2015.csv"), parse_dates=[0])
df3=pd.read_csv(path.data_raw_dir("SPY2016.csv"), parse_dates=[0])
df4=pd.read_csv(path.data_raw_dir("SPY2017.csv"), parse_dates=[0])
df5=pd.read_csv(path.data_raw_dir("SPY2018.csv"), parse_dates=[0])
df6=pd.read_csv(path.data_raw_dir("SPY2019.csv"), parse_dates=[0])
df7=pd.read_csv(path.data_raw_dir("SPY2020.csv"), parse_dates=[0])

Concatenamos en un solo DataFrame

In [97]:
df=pd.concat([df1,df2,df3,df4,df5,df6,df7],ignore_index=True)
pd.options.display.float_format='{:,.2f}'.format

Separamos Año, Mes, Dia, Hora, Minutos y Segundos

In [98]:
df['day']=df['DateTime'].dt.day
df['month']=df['DateTime'].dt.month
df['year']=df['DateTime'].dt.year
df['hour']=df['DateTime'].dt.hour
df['minute']=df['DateTime'].dt.minute
df['DayWeek']=df['DateTime'].dt.strftime("%A")

df.convert_dtypes().dtypes

DateTime    datetime64[ns]
Open               Float64
High               Float64
Low                Float64
Close              Float64
day                  Int64
month                Int64
year                 Int64
hour                 Int64
minute               Int64
DayWeek             string
dtype: object

Revisamos como va quedando nuestro DataFrame

In [99]:
df.head(5)

,DateTime,Open,High,Low,Close,day,month,year,hour,minute,DayWeek
0,2014-01-02 09:30:00,183.97,184.06,183.29,183.55,2,1,2014,9,30,Thursday
1,2014-01-02 10:00:00,183.55,183.65,183.12,183.44,2,1,2014,10,0,Thursday
2,2014-01-02 10:30:00,183.44,183.68,183.31,183.47,2,1,2014,10,30,Thursday
3,2014-01-02 11:00:00,183.47,183.61,183.34,183.44,2,1,2014,11,0,Thursday
4,2014-01-02 11:30:00,183.44,183.46,182.86,183.02,2,1,2014,11,30,Thursday


Obtenemos la descripcion de las Columnas

In [100]:
df.columns

Index(['DateTime', 'Open', 'High', 'Low', 'Close', 'day', 'month', 'year',
       'hour', 'minute', 'DayWeek'],
      dtype='object')

Creamos un Dataframe de 30 minutos para mantener separados los calculos

In [101]:
df30=df[['DateTime','year','month','day','hour','minute','DayWeek','Open','High','Low','Close']]
df30.to_csv(path.data_processed_dir("SPYV30M.csv"), index = False)

Empezamos a crear el DataFrame de temporalidad 1H

In [102]:
df['min30']=df['minute']==30
df.loc[ (df['hour'] != 15) & (df['minute'] == 30), 'HighT'] =df['High']>=df['High'].shift(- 1)
df.loc[ (df['hour'] != 15) & (df['minute'] == 30), 'HighL'] =df['High']<df['High'].shift(- 1)
df.loc[ (df['hour'] != 15) & (df['minute'] == 30), 'LowT'] =df['Low']>=df['Low'].shift(- 1)
df.loc[ (df['hour'] != 15) & (df['minute'] == 30), 'LowL'] =df['Low']<df['Low'].shift(- 1)

df['OpenOk']=df['min30']*df['Open']  
df['HighOk']=df['HighT']*df['High']

df.loc[(df['hour'] == 15) & (df['minute'] == 30), 'HighOk'] =  df['min30']*df['High']  
df.loc[(df['hour'] != 15) & (df['minute'] == 30)& (df['HighL'] == True), 'HighOk'] =  df['HighL']*df['High'].shift(-1)
df['LowOk']=df['LowL']*df['Low']
df.loc[(df['hour'] == 15) & (df['minute'] == 30), 'LowOk'] =  df['min30']*df['Low']  
df.loc[(df['hour'] != 15) & (df['minute'] == 30)& (df['LowT'] == True), 'LowOk'] =  df['LowT']*df['Low'].shift(-1)
df.loc[(df['hour'] == 15) & (df['minute'] == 30), 'Closeok'] =  df['min30']*df['Close'] 
df.loc[(df['hour'] != 15) & (df['minute'] == 30), 'Closeok'] =  df['min30']*df['Close'].shift(-1)
df1H=df
df1H.drop(index=df1H.loc[df1H['min30']==False].index ,columns=['Open','High','Low','Close'], inplace=True)

Creamos un Dataframe de Temporalidad 1 Hora y guardamos en un archivo .csv

In [103]:
df1H=df1H[['DateTime','year','month','day','hour','minute','DayWeek','OpenOk','HighOk','LowOk','Closeok']]
df1H.to_csv(path.data_processed_dir("SPYV1H.csv"), index = False)
df1H.tail(7)

,DateTime,year,month,day,hour,minute,DayWeek,OpenOk,HighOk,LowOk,Closeok
22810,2020-12-31 09:30:00,2020,12,31,9,30,Thursday,371.78,372.23,371.22,371.94
22812,2020-12-31 10:30:00,2020,12,31,10,30,Thursday,371.94,372.04,371.52,371.69
22814,2020-12-31 11:30:00,2020,12,31,11,30,Thursday,371.69,371.82,371.28,371.81
22816,2020-12-31 12:30:00,2020,12,31,12,30,Thursday,371.81,372.08,371.68,371.84
22818,2020-12-31 13:30:00,2020,12,31,13,30,Thursday,371.84,372.62,371.46,372.43
22820,2020-12-31 14:30:00,2020,12,31,14,30,Thursday,372.43,373.63,372.43,373.32
22822,2020-12-31 15:30:00,2020,12,31,15,30,Thursday,373.32,374.65,373.05,373.86


Empezamos con el calculo del DataFrame en temporalidad 1 dia.
Calculamos los valores de apertura en temporalidad 1D

In [104]:
c=df1H.groupby(['year','month','day','DayWeek'])['OpenOk'].first()
c=c.reset_index()
c

,year,month,day,DayWeek,OpenOk
0,2014,1,2,Thursday,183.97
1,2014,1,3,Friday,183.19
2,2014,1,6,Monday,183.46
3,2014,1,7,Tuesday,183.09
4,2014,1,8,Wednesday,183.45
...,...,...,...,...,...
1758,2020,12,24,Thursday,368.05
1759,2020,12,28,Monday,371.75
1760,2020,12,29,Tuesday,373.81
1761,2020,12,30,Wednesday,372.33


Calculamos los valores de cierre en temporalidad 1D

In [105]:
d=df1H.groupby(['year','month','day','DayWeek'])['Closeok'].last()
d=d.reset_index()
d

,year,month,day,DayWeek,Closeok
0,2014,1,2,Thursday,182.94
1,2014,1,3,Friday,182.79
2,2014,1,6,Monday,182.39
3,2014,1,7,Tuesday,183.44
4,2014,1,8,Wednesday,183.53
...,...,...,...,...,...
1758,2020,12,24,Thursday,371.37
1759,2020,12,28,Monday,372.13
1760,2020,12,29,Tuesday,371.47
1761,2020,12,30,Wednesday,371.92


Calculamos los valores del alto en temporalidad 1D

In [106]:
a=df1H.groupby(['year','month','day','DayWeek'])['HighOk'].max()
a=a.reset_index()
a

,year,month,day,DayWeek,HighOk
0,2014,1,2,Thursday,184.06
1,2014,1,3,Friday,183.59
2,2014,1,6,Monday,183.55
3,2014,1,7,Tuesday,183.78
4,2014,1,8,Wednesday,183.82
...,...,...,...,...,...
1758,2020,12,24,Thursday,371.81
1759,2020,12,28,Monday,372.58
1760,2020,12,29,Tuesday,373.99
1761,2020,12,30,Wednesday,373.07


Calculamos los valores del bajo en temporalidad 1D

In [107]:
b=df1H.groupby(['year','month','day','DayWeek'])['LowOk'].min()
b=b.reset_index()
b

,year,month,day,DayWeek,LowOk
0,2014,1,2,Thursday,182.48
1,2014,1,3,Friday,182.63
2,2014,1,6,Monday,182.08
3,2014,1,7,Tuesday,182.95
4,2014,1,8,Wednesday,182.89
...,...,...,...,...,...
1758,2020,12,24,Thursday,367.45
1759,2020,12,28,Monday,371.07
1760,2020,12,29,Tuesday,370.82
1761,2020,12,30,Wednesday,371.57


Creamos un Dataframe temporalidad 1D y empezamos a unir cada columna

In [108]:
df1D=pd.merge(c,a)
df1D=pd.merge(df1D,b)
df1D=pd.merge(df1D,d)
df1D


,year,month,day,DayWeek,OpenOk,HighOk,LowOk,Closeok
0,2014,1,2,Thursday,183.97,184.06,182.48,182.94
1,2014,1,3,Friday,183.19,183.59,182.63,182.79
2,2014,1,6,Monday,183.46,183.55,182.08,182.39
3,2014,1,7,Tuesday,183.09,183.78,182.95,183.44
4,2014,1,8,Wednesday,183.45,183.82,182.89,183.53
...,...,...,...,...,...,...,...,...
1758,2020,12,24,Thursday,368.05,371.81,367.45,371.37
1759,2020,12,28,Monday,371.75,372.58,371.07,372.13
1760,2020,12,29,Tuesday,373.81,373.99,370.82,371.47
1761,2020,12,30,Wednesday,372.33,373.07,371.57,371.92


Definimos una funcion donde colocamos una bandera para determinar la variacion diaria si es mayor a 0 colocamos 1 ,si es menor a cero colocamos -1 y si es 0 colocamos 0

In [109]:
def fun(x):
    if(x>0):
        return 1
    if (x<0):
        return -1
    if (x==0):
        return 0


Realizamos los calculos de indicadores (Ma200,MA50,EMA8, EMA21, ) en temporalidad , y realizamos un analisis del precio respecto a cada indicador:
    . Precio sobre EMA8
    . EMA8 sobre EMA21
    . EMA21 sobre MA50
    . MA50 sobre MA200

In [110]:
df1D['%D']=round((df1D['Closeok']/df1D['OpenOk']-1)*100,7)
df1D['DayStatus']=df1D['%D'].apply(fun)
df1D['MA200']=round(df1D['Closeok'].rolling(window=200).mean(),2)
df1D['MA50']=round(df1D['Closeok'].rolling(window=50).mean(),2)
df1D['EMA21']=round(pd.Series.ewm(df1D['Closeok'], span=21).mean(),2)
df1D['EMA8']=round(pd.Series.ewm(df1D['Closeok'], span=8).mean(),2)
df1D['P/8']=df1D['Closeok']>=df1D['EMA8']
df1D['8/21']=df1D['EMA8']>df1D['EMA21']
df1D['21/50']=df1D['EMA21']>df1D['MA50']
df1D['50/200']=df1D['MA50']>df1D['MA200']
df1D

,year,month,day,DayWeek,OpenOk,HighOk,LowOk,Closeok,%D,DayStatus,MA200,MA50,EMA21,EMA8,P/8,8/21,21/50,50/200
0,2014,1,2,Thursday,183.97,184.06,182.48,182.94,-0.56,-1,NaN,NaN,182.94,182.94,True,False,False,False
1,2014,1,3,Friday,183.19,183.59,182.63,182.79,-0.22,-1,NaN,NaN,182.86,182.86,False,False,False,False
2,2014,1,6,Monday,183.46,183.55,182.08,182.39,-0.58,-1,NaN,NaN,182.69,182.66,False,False,False,False
3,2014,1,7,Tuesday,183.09,183.78,182.95,183.44,0.19,1,NaN,NaN,182.90,182.93,True,True,False,False
4,2014,1,8,Wednesday,183.45,183.82,182.89,183.53,0.04,1,NaN,NaN,183.05,183.12,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1758,2020,12,24,Thursday,368.05,371.81,367.45,371.37,0.90,1,320.73,355.83,366.08,368.84,True,True,True,True
1759,2020,12,28,Monday,371.75,372.58,371.07,372.13,0.10,1,321.36,356.29,366.63,369.57,True,True,True,True
1760,2020,12,29,Tuesday,373.81,373.99,370.82,371.47,-0.63,-1,322.03,356.75,367.07,369.99,True,True,True,True
1761,2020,12,30,Wednesday,372.33,373.07,371.57,371.92,-0.11,-1,322.68,357.29,367.51,370.42,True,True,True,True


Borramos datos de los dias donde la variacion da cero

In [111]:
df1D.drop(index=df1D.loc[df1D['DayStatus']==0].index , inplace=True)

Calculamos los rangos de variacion diari del precio

In [112]:
pd.cut(df1D['%D'],bins=3).value_counts()

(-3.181, 1.495]     1675
(1.495, 6.171]        71
(-7.871, -3.181]      10
Name: %D, dtype: int64

Calculamos mas indicadores:
    .ATR
    .MACD
    .Engulfing
    .DCC
    .Doji
    .RSI

In [113]:
df1D['ATRREAL'] = round( talib.ATR(df1D['HighOk'], df1D['LowOk'], df1D['Closeok'], timeperiod=14),2)
macd, macdsignal, macdhist = talib.MACD(df1D['Closeok'], fastperiod=8, slowperiod=21, signalperiod=9)
df1D["MACDHIST"]=round(macdhist,2)
df1D["MACD"]=round(macd,2)
df1D['MACDSignal']=round(macdsignal,2)
df1D['Engulfing'] =  talib.CDLENGULFING(df1D['OpenOk'], df1D['HighOk'], df1D['LowOk'], df1D['Closeok'])
df1D['DCC'] = talib.CDLDARKCLOUDCOVER(df1D['OpenOk'], df1D['HighOk'], df1D['LowOk'], df1D['Closeok'], penetration=0)
df1D['Doji'] = talib.CDLDOJI(df1D['OpenOk'], df1D['HighOk'], df1D['LowOk'], df1D['Closeok'])
df1D['IHAMER'] = talib.CDLINVERTEDHAMMER(df1D['OpenOk'], df1D['HighOk'], df1D['LowOk'], df1D['Closeok'])
df1D['HAMER'] = talib.CDLHAMMER(df1D['OpenOk'], df1D['HighOk'], df1D['LowOk'], df1D['Closeok'])
df1D['2Crows'] = talib.CDL2CROWS(df1D['OpenOk'], df1D['HighOk'], df1D['LowOk'], df1D['Closeok'])
df1D['3BCrows'] = talib.CDL3BLACKCROWS(df1D['OpenOk'], df1D['HighOk'], df1D['LowOk'], df1D['Closeok'])
df1D['Momentum'] = round(talib.MOM(df1D['Closeok'], timeperiod=10),2)
df1D['RSI'] = round(talib.RSI(df1D['Closeok'], timeperiod=14),2)

Agragamos el formato de numeros decimales

In [114]:
pd.options.display.float_format='{:,.2f}'.format

Agregamos dos columnas indicando el estatus del dia anterior y el estatus de tras anterior

In [115]:
df1D['BackStatus']=df1D['DayStatus'].shift(1)
df1D['TBackStatus']=df1D['DayStatus'].shift(2)

In [116]:
##df1D['ones'] = 1
##df1D['Bins_ATR']=pd.cut(df1D['ATRREAL'],bins=[1,4.5,7,10,13,16])
##df1D['Bins_RSI']=pd.cut(df1D['RSI'],bins=[15,35,47,60,72,90])
#df1D['Bins_%D']=pd.cut(df1D['%D'],bins=[-5,-2.8,-1.2,-1,-0.5,0,0.5,1,1.9,4])

Guardamos Dataframe en archivo .csv para futuros analisis

In [117]:
df1D.to_csv(path.data_processed_dir("SPYVD.csv"), index = False)
df1D[(df1D['year']>=2018)].head(20)

,year,month,day,DayWeek,OpenOk,HighOk,LowOk,Closeok,%D,DayStatus,...,DCC,Doji,IHAMER,HAMER,2Crows,3BCrows,Momentum,RSI,BackStatus,TBackStatus
1007,2018,1,2,Tuesday,267.84,268.80,267.39,268.79,0.35,1,...,0,0,0,0,0,0,2.27,68.39,-1.00,1.00
1008,2018,1,3,Wednesday,268.96,270.63,268.96,270.46,0.56,1,...,0,0,0,0,0,0,2.29,72.95,1.00,-1.00
1009,2018,1,4,Thursday,271.21,272.15,271.10,271.63,0.15,1,...,0,0,0,0,0,0,4.44,75.61,1.00,1.00
1010,2018,1,5,Friday,272.50,273.56,271.94,273.39,0.33,1,...,0,0,0,0,0,0,6.36,78.96,1.00,1.00
1011,2018,1,8,Monday,273.31,274.09,272.98,273.93,0.23,1,...,0,0,0,0,0,0,6.37,79.87,1.00,1.00
1012,2018,1,9,Tuesday,274.39,275.24,274.08,274.52,0.05,1,...,0,0,0,0,0,0,7.04,80.85,1.00,1.00
1013,2018,1,10,Wednesday,273.67,274.42,272.92,274.10,0.16,1,...,0,0,0,0,0,0,6.94,77.95,1.00,1.00
1014,2018,1,11,Thursday,274.73,276.08,274.56,276.08,0.49,1,...,0,0,0,0,0,0,8.75,81.35,1.00,1.00
1015,2018,1,12,Friday,276.39,278.10,276.27,277.91,0.55,1,...,0,0,0,0,0,0,10.05,83.83,1.00,1.00
1016,2018,1,16,Tuesday,279.35,280.08,276.18,276.96,-0.86,-1,...,-100,0,0,0,0,0,10.08,78.03,1.00,1.00


Revisamos conteo de datos por año

In [118]:
df1D.groupby(['year'])['month'].count()

year
2014    250
2015    252
2016    250
2017    250
2018    250
2019    251
2020    253
Name: month, dtype: int64

In [119]:
df1D.convert_dtypes().dtypes

year             Int64
month            Int64
day              Int64
DayWeek         string
OpenOk         Float64
HighOk         Float64
LowOk          Float64
Closeok        Float64
%D             Float64
DayStatus        Int64
MA200          Float64
MA50           Float64
EMA21          Float64
EMA8           Float64
P/8            boolean
8/21           boolean
21/50          boolean
50/200         boolean
ATRREAL        Float64
MACDHIST       Float64
MACD           Float64
MACDSignal     Float64
Engulfing        Int32
DCC              Int32
Doji             Int32
IHAMER           Int32
HAMER            Int32
2Crows           Int32
3BCrows          Int32
Momentum       Float64
RSI            Float64
BackStatus       Int64
TBackStatus      Int64
dtype: object